In [ ]:
# pip install customized_konlpy

In [1]:
import pandas as pd
df=pd.read_csv('줄임말리스트.csv')
df

,줄임말
0,평냉
1,비냉
2,물냉
3,완냉
4,찍먹
...,...
94,레게노
95,레전드
96,재방문
97,캐주얼


In [2]:
li=[]
for i in range(len(df['줄임말'])):
#     print(df['줄임말'][i])
    li.append(df['줄임말'][i])
# li

In [3]:
re=pd.read_csv('0923_DGY_리뷰전처리.csv')
re.head()

,상호ID,리뷰
0,S0000002,점심식사분위기캐주얼한매우만족만족보통불만매우불만평가 참여율이 부족합니다 평가에 참여해...
1,S0000003,식사모임 기념일 실버푸드분위기서민적인 경관야경이 좋은 숨은맛집편의시설무료주차 개별룸...
2,S0000004,사당나인피스그릴 삼겹살에 버터 조합이라니언제나 먹어도 맛있는 삼겹살 사당역 맛집을...
3,S0000005,회식 술모임 접대 가족외식분위기지역주민이 찾는 숨은맛집 가성비좋은편의시설칸막이 무료...
4,S0000008,저녁식사 술모임분위기가성비좋은 서민적인매우만족만족보통불만매우불만평가 참여율이 부족합...


In [4]:
with open('불용어리스트.txt', "r", encoding='UTF-8') as file:
    strings = file.readlines()
    #print(strings)

In [5]:
불용어=pd.DataFrame({'불용어':strings})
불용어=불용어['불용어'].str.replace('\n','')
stopwords=불용어.to_list()

In [6]:
import urllib.request
from konlpy.tag import Okt
import pandas as pd
import matplotlib.pyplot as plt
from ckonlpy.tag import Twitter
twitter = Twitter()

C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [7]:
twitter.add_dictionary(li, 'Noun')

In [8]:
tokenized_data = []
for sentence in re['리뷰']:
    temp_X = twitter.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)
#     print('불용어=>',temp_X)

In [9]:
len(tokenized_data)

20503

In [12]:
tokenized_data[0]

['점심식사',
 '분위기',
 '캐주얼',
 '한',
 '매우',
 '만족',
 '만',
 '족',
 '보통',
 '불만',
 '매우',
 '불만',
 '평가',
 '참여',
 '율',
 '부족',
 '하다',
 '평가',
 '참여',
 '해주',
 '세요',
 '홍보',
 '비방',
 '부적절',
 '한',
 '평가',
 '평점',
 '산정',
 '제외',
 '되다',
 '맛집',
 '평가',
 '하고',
 '원',
 '적립',
 '받다',
 '기초',
 '코콘',
 '곳',
 '작성',
 '개',
 '년',
 '맛',
 '가격',
 '서비스',
 '더블',
 '사케',
 '동',
 '연어',
 '두툼',
 '하고',
 '맛남',
 '와사비',
 '더',
 '달라',
 '니까',
 '만큼',
 '이나',
 '주심',
 '최고',
 '점심식사',
 '캐주얼',
 '한',
 '공감']

In [11]:
import numpy as np
words = np.hstack(tokenized_data)

In [13]:
from collections import Counter

words_counter = Counter(words)
words_counter = words_counter.most_common()

In [14]:
from eunjeon import Mecab
m = Mecab()

In [15]:
words_counter_pos = []
for item in words_counter:
  pos_tuple = (item[0], item[1], m.pos(item[0]))
  words_counter_pos.append(pos_tuple)

In [16]:
words_counter_pos[:5]

[('하다', 125942, [('하', 'VV'), ('다', 'EC')]),
 ('맛', 114767, [('맛', 'NNG')]),
 ('한', 103252, [('한', 'MM')]),
 ('곳', 84106, [('곳', 'NNG')]),
 ('가격', 76116, [('가격', 'NNG')])]

In [17]:
wcp_df = pd.DataFrame(words_counter_pos)
wcp_df.columns = ['word', 'count', 'pos']

In [18]:
wcp_df

,word,count,pos
0,하다,125942,"[(하, VV), (다, EC)]"
1,맛,114767,"[(맛, NNG)]"
2,한,103252,"[(한, MM)]"
3,곳,84106,"[(곳, NNG)]"
4,가격,76116,"[(가격, NNG)]"
...,...,...,...
40868,왜그,1,"[(왜, MAG), (그, MM)]"
40869,함또,1,"[(함, MAG), (또, MAG)]"
40870,사나이,1,"[(사나이, NNG)]"
40871,듭니다제,1,"[(듭니다제, UNKNOWN)]"


In [19]:
pos_analysis_df =wcp_df[wcp_df['word'].map(len) >= 2]
pos_analysis_df['pos'] = pos_analysis_df['pos'].apply(lambda x : x[0][1])

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
pos_analysis_df

,word,count,pos
0,하다,125942,VV
4,가격,76116,NNG
5,먹다,73358,VV
6,좋다,72230,VA
8,서비스,64220,NNG
...,...,...,...
40868,왜그,1,MAG
40869,함또,1,MAG
40870,사나이,1,NNG
40871,듭니다제,1,UNKNOWN


In [21]:
pos_analysis_df['pos'].value_counts()

NNG              21489
NNP               7697
VV                1974
MAG               1835
IC                 875
                 ...  
XSA+EC+VX+ETM        1
VA+EF                1
EP+ETN               1
MAG+VCP+ETM          1
JKQ+JX               1
Name: pos, Length: 119, dtype: int64

In [22]:
w=pd.DataFrame({'word':tokenized_data})
w

,word
0,"[점심식사, 분위기, 캐주얼, 한, 매우, 만족, 만, 족, 보통, 불만, 매우, ..."
1,"[식사, 모임, 기념일, 실버, 푸드, 분위기, 서민적인, 경관, 야경, 좋다, 숨..."
2,"[사당, 나인, 피스, 그릴, 삼겹살, 버터, 조합, 이라니, 먹다, 어도, 맛있다..."
3,"[회식, 술, 모임, 접대, 가족, 외식, 분위기, 지, 역, 주민, 찾다, 숨다,..."
4,"[저녁식사, 술, 모임, 분위기, 성비, 좋다, 서민적인, 매우, 만, 족, 만족,..."
...,...
20498,"[송파구, 맛집, 여름, 엔, 냉면, 이지, 함흥, 평양, 냉면, 전문점, 당, 고..."
20499,"[방이, 샤브샤브, 칼국수, 대치동, 칼국수, 진성, 칼국수, 청담동, 칼국수, 보..."
20500,"[저녁식사, 실버, 푸드, 데이트, 분위기, 지, 역, 주민, 찾다, 푸다, 짐, ..."
20501,"[점심식사, 분위기, 고, 급, 스러운, 예쁘다, 푸다, 짐, 한, 편의, 시설, ..."


In [ ]:
# w=w.iloc[:10,:]

In [ ]:
# def word_one():
#     dy=[]
#     for i in w['word'][1]:
#         if len(i)>1:
#             dy.append(i)
# #             print(i)
#     return dy

In [ ]:
# word_one()

In [23]:
gy=[]
for j in range(len(w['word'])):
    dy=[]
    for i in w['word'][j]:
        if len(i)>1:
            dy.append(i)
    gy.append(dy)
# print(gy)

In [24]:
len(gy)

20503

In [25]:
gy[0]

['점심식사',
 '분위기',
 '캐주얼',
 '매우',
 '만족',
 '보통',
 '불만',
 '매우',
 '불만',
 '평가',
 '참여',
 '부족',
 '하다',
 '평가',
 '참여',
 '해주',
 '세요',
 '홍보',
 '비방',
 '부적절',
 '평가',
 '평점',
 '산정',
 '제외',
 '되다',
 '맛집',
 '평가',
 '하고',
 '적립',
 '받다',
 '기초',
 '코콘',
 '작성',
 '가격',
 '서비스',
 '더블',
 '사케',
 '연어',
 '두툼',
 '하고',
 '맛남',
 '와사비',
 '달라',
 '니까',
 '만큼',
 '이나',
 '주심',
 '최고',
 '점심식사',
 '캐주얼',
 '공감']

### 위에서 분석된것들을 미리 만들어놓은 키워드,목적,대표메뉴사전과 비교하여 분류한다.

In [26]:
dic=pd.read_csv('0923_사전리스트최종.csv')
dic

,단어,a,b,c,형태소,분류,종성,읽기,1,2,3,4,5
0,매콤,NaN,NaN,NaN,NNP,먹찌s pick,T,매콤,*,*,*,*,*
1,쏘다,NaN,NaN,NaN,NNP,먹찌s pick,F,쏘다,*,*,*,*,*
2,퀄리티,NaN,NaN,NaN,NNP,먹찌s pick,F,퀄리티,*,*,*,*,*
3,메인,NaN,NaN,NaN,NNP,먹찌s pick,T,메인,*,*,*,*,*
4,존맛,NaN,NaN,NaN,NNP,먹찌s pick,T,존맛,*,*,*,*,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,레게노,NaN,NaN,NaN,NNP,먹찌s pick,T,레게노,*,*,*,*,*
741,레전드,NaN,NaN,NaN,NNP,먹찌s pick,T,레전드,*,*,*,*,*
742,재방문,NaN,NaN,NaN,NNP,먹찌s pick,F,재방문,*,*,*,*,*
743,캐주얼,NaN,NaN,NaN,NNP,먹찌s pick,F,캐주얼,*,*,*,*,*


In [27]:
dic=dic[['단어','분류']]
dic

,단어,분류
0,매콤,먹찌s pick
1,쏘다,먹찌s pick
2,퀄리티,먹찌s pick
3,메인,먹찌s pick
4,존맛,먹찌s pick
...,...,...
740,레게노,먹찌s pick
741,레전드,먹찌s pick
742,재방문,먹찌s pick
743,캐주얼,먹찌s pick


In [28]:
먹찌s=dic.loc[dic['분류']=='먹찌s pick']
먹찌s

,단어,분류
0,매콤,먹찌s pick
1,쏘다,먹찌s pick
2,퀄리티,먹찌s pick
3,메인,먹찌s pick
4,존맛,먹찌s pick
...,...,...
740,레게노,먹찌s pick
741,레전드,먹찌s pick
742,재방문,먹찌s pick
743,캐주얼,먹찌s pick


In [29]:
먹찌s=먹찌s['단어'].tolist()

In [30]:
목적=dic.loc[dic['분류']=='목적']
목적

,단어,분류
288,테이크아웃,목적
289,무한리필,목적
290,피크닉,목적
291,가족,목적
292,데이트,목적
...,...,...
367,벙개,목적
368,비건,목적
682,얼리어먹터,목적
687,혼술,목적


In [31]:
목적=목적['단어'].tolist()

In [32]:
중분류=pd.read_csv('HJ_중분류테이블.csv')
중분류

,중분류ID,중분류명
0,J00001,LA갈비
1,J00002,갈비찜
2,J00003,갈비탕
3,J00004,곱창
4,J00005,곱창볶음
...,...,...
425,J00426,러시아
426,J00427,맥엔치즈
427,J00428,브리또
428,J00429,퓨전술집


In [33]:
중분류=중분류['중분류명'].tolist()

In [34]:
key=pd.DataFrame(columns={'목적','대표메뉴','먹찌s 키워드'})
key

,목적,먹찌s 키워드,대표메뉴


In [35]:
for num in range(len(gy)):
    purpose=[]
    keyword=[]
    menu=[]
    for i in gy[num]:
        if i in 목적:
            a=i
            purpose.append(a)
        elif i in 먹찌s:
            b=i
            keyword.append(b)
        elif i in 중분류:
            c=i
            menu.append(c)
    keyword=set(keyword)
    purpose=set(purpose)
    menu=set(menu)
    keyword=list(keyword)
    purpose=list(purpose)
    menu=list(menu)
    key.loc[num]=[purpose,keyword,menu]

In [36]:
key

,목적,먹찌s 키워드,대표메뉴
0,[점심식사],"[두툼, 맛집, 캐주얼]",[연어]
1,"[접대, 가족, 기념일, 점심식사, 모임]","[맛있다, 맛집, 좋다, 가성비, 서민적인]","[소갈비, 돼지갈비, 갈비탕]"
2,"[점심, 모임]","[고급, 추천, 맛있다, 맛집, 빨갛다, 좋다, 굽다, 유명, 존맛탱, 존맛]","[삼겹살, 스테이크, 돼지고기]"
3,"[접대, 회식, 가족, 혼밥, 모임]","[물냉, 짭짤하다, 달달, 적다, 맛있다, 맛집, 좋다, 비냉, 가성비, 서민적인]","[냉면, 등심, 돼지갈비]"
4,[모임],"[괜찮다, 맛있다, 맛집, 좋다, 가성비, 서민적인]","[갈비, 돼지갈비]"
...,...,...,...
20498,[점심],"[시원, 평범, 달콤, 짜다, 맛집]","[갈비찜, 냉면]"
20499,[],[],"[샤브샤브, 보쌈, 칼국수]"
20500,"[점심식사, 데이트]","[비싸다, 시끌벅적, 맛있다, 맛집, 쏘쏘, 나쁘다, 서민적인]",[파전]
20501,[점심식사],"[맛집, 많이, 예쁘다, 고급]",[]


In [37]:
re_table = pd.concat([re, key], axis=1)
re_table

,상호ID,리뷰,목적,먹찌s 키워드,대표메뉴
0,S0000002,점심식사분위기캐주얼한매우만족만족보통불만매우불만평가 참여율이 부족합니다 평가에 참여해...,[점심식사],"[두툼, 맛집, 캐주얼]",[연어]
1,S0000003,식사모임 기념일 실버푸드분위기서민적인 경관야경이 좋은 숨은맛집편의시설무료주차 개별룸...,"[접대, 가족, 기념일, 점심식사, 모임]","[맛있다, 맛집, 좋다, 가성비, 서민적인]","[소갈비, 돼지갈비, 갈비탕]"
2,S0000004,사당나인피스그릴 삼겹살에 버터 조합이라니언제나 먹어도 맛있는 삼겹살 사당역 맛집을...,"[점심, 모임]","[고급, 추천, 맛있다, 맛집, 빨갛다, 좋다, 굽다, 유명, 존맛탱, 존맛]","[삼겹살, 스테이크, 돼지고기]"
3,S0000005,회식 술모임 접대 가족외식분위기지역주민이 찾는 숨은맛집 가성비좋은편의시설칸막이 무료...,"[접대, 회식, 가족, 혼밥, 모임]","[물냉, 짭짤하다, 달달, 적다, 맛있다, 맛집, 좋다, 비냉, 가성비, 서민적인]","[냉면, 등심, 돼지갈비]"
4,S0000008,저녁식사 술모임분위기가성비좋은 서민적인매우만족만족보통불만매우불만평가 참여율이 부족합...,[모임],"[괜찮다, 맛있다, 맛집, 좋다, 가성비, 서민적인]","[갈비, 돼지갈비]"
...,...,...,...,...,...
20498,S0034701,송파구 맛집 여름엔 냉면이지 함흥평양냉면 전문점 당고개냉면너무 덥다 정말 너무 더...,[점심],"[시원, 평범, 달콤, 짜다, 맛집]","[갈비찜, 냉면]"
20499,S0034702,방이샤브샤브칼국수대치동 칼국수 진성칼국수청담동 칼국수 보쌈정식 어바웃샤브샤브...,[],[],"[샤브샤브, 보쌈, 칼국수]"
20500,S0034703,저녁식사 실버푸드 데이트분위기지역주민이 찾는 푸짐한 숨은맛집편의시설발렛주차매우만족만...,"[점심식사, 데이트]","[비싸다, 시끌벅적, 맛있다, 맛집, 쏘쏘, 나쁘다, 서민적인]",[파전]
20501,S0034704,점심식사분위기고급스러운 예쁜 푸짐한편의시설무료주차매우만족만족보통불만매우불만평가참여를...,[점심식사],"[맛집, 많이, 예쁘다, 고급]",[]


In [38]:
re_table.to_csv('DGY_리뷰테이블4.csv', index=False, encoding='utf-8-sig')

In [39]:
table=pd.read_csv('DGY_리뷰테이블4.csv')
table

,상호ID,리뷰,목적,먹찌s 키워드,대표메뉴
0,S0000002,점심식사분위기캐주얼한매우만족만족보통불만매우불만평가 참여율이 부족합니다 평가에 참여해...,['점심식사'],"['두툼', '맛집', '캐주얼']",['연어']
1,S0000003,식사모임 기념일 실버푸드분위기서민적인 경관야경이 좋은 숨은맛집편의시설무료주차 개별룸...,"['접대', '가족', '기념일', '점심식사', '모임']","['맛있다', '맛집', '좋다', '가성비', '서민적인']","['소갈비', '돼지갈비', '갈비탕']"
2,S0000004,사당나인피스그릴 삼겹살에 버터 조합이라니언제나 먹어도 맛있는 삼겹살 사당역 맛집을...,"['점심', '모임']","['고급', '추천', '맛있다', '맛집', '빨갛다', '좋다', '굽다', '...","['삼겹살', '스테이크', '돼지고기']"
3,S0000005,회식 술모임 접대 가족외식분위기지역주민이 찾는 숨은맛집 가성비좋은편의시설칸막이 무료...,"['접대', '회식', '가족', '혼밥', '모임']","['물냉', '짭짤하다', '달달', '적다', '맛있다', '맛집', '좋다', ...","['냉면', '등심', '돼지갈비']"
4,S0000008,저녁식사 술모임분위기가성비좋은 서민적인매우만족만족보통불만매우불만평가 참여율이 부족합...,['모임'],"['괜찮다', '맛있다', '맛집', '좋다', '가성비', '서민적인']","['갈비', '돼지갈비']"
...,...,...,...,...,...
20498,S0034701,송파구 맛집 여름엔 냉면이지 함흥평양냉면 전문점 당고개냉면너무 덥다 정말 너무 더...,['점심'],"['시원', '평범', '달콤', '짜다', '맛집']","['갈비찜', '냉면']"
20499,S0034702,방이샤브샤브칼국수대치동 칼국수 진성칼국수청담동 칼국수 보쌈정식 어바웃샤브샤브...,[],[],"['샤브샤브', '보쌈', '칼국수']"
20500,S0034703,저녁식사 실버푸드 데이트분위기지역주민이 찾는 푸짐한 숨은맛집편의시설발렛주차매우만족만...,"['점심식사', '데이트']","['비싸다', '시끌벅적', '맛있다', '맛집', '쏘쏘', '나쁘다', '서민적인']",['파전']
20501,S0034704,점심식사분위기고급스러운 예쁜 푸짐한편의시설무료주차매우만족만족보통불만매우불만평가참여를...,['점심식사'],"['맛집', '많이', '예쁘다', '고급']",[]


In [40]:
table.loc[table['먹찌s 키워드']=="[]",'먹찌s 키워드']=None

In [41]:
table.loc[table['목적']=="[]",'목적']=None
table.loc[table['대표메뉴']=="[]",'대표메뉴']=None

In [42]:
table

,상호ID,리뷰,목적,먹찌s 키워드,대표메뉴
0,S0000002,점심식사분위기캐주얼한매우만족만족보통불만매우불만평가 참여율이 부족합니다 평가에 참여해...,['점심식사'],"['두툼', '맛집', '캐주얼']",['연어']
1,S0000003,식사모임 기념일 실버푸드분위기서민적인 경관야경이 좋은 숨은맛집편의시설무료주차 개별룸...,"['접대', '가족', '기념일', '점심식사', '모임']","['맛있다', '맛집', '좋다', '가성비', '서민적인']","['소갈비', '돼지갈비', '갈비탕']"
2,S0000004,사당나인피스그릴 삼겹살에 버터 조합이라니언제나 먹어도 맛있는 삼겹살 사당역 맛집을...,"['점심', '모임']","['고급', '추천', '맛있다', '맛집', '빨갛다', '좋다', '굽다', '...","['삼겹살', '스테이크', '돼지고기']"
3,S0000005,회식 술모임 접대 가족외식분위기지역주민이 찾는 숨은맛집 가성비좋은편의시설칸막이 무료...,"['접대', '회식', '가족', '혼밥', '모임']","['물냉', '짭짤하다', '달달', '적다', '맛있다', '맛집', '좋다', ...","['냉면', '등심', '돼지갈비']"
4,S0000008,저녁식사 술모임분위기가성비좋은 서민적인매우만족만족보통불만매우불만평가 참여율이 부족합...,['모임'],"['괜찮다', '맛있다', '맛집', '좋다', '가성비', '서민적인']","['갈비', '돼지갈비']"
...,...,...,...,...,...
20498,S0034701,송파구 맛집 여름엔 냉면이지 함흥평양냉면 전문점 당고개냉면너무 덥다 정말 너무 더...,['점심'],"['시원', '평범', '달콤', '짜다', '맛집']","['갈비찜', '냉면']"
20499,S0034702,방이샤브샤브칼국수대치동 칼국수 진성칼국수청담동 칼국수 보쌈정식 어바웃샤브샤브...,None,None,"['샤브샤브', '보쌈', '칼국수']"
20500,S0034703,저녁식사 실버푸드 데이트분위기지역주민이 찾는 푸짐한 숨은맛집편의시설발렛주차매우만족만...,"['점심식사', '데이트']","['비싸다', '시끌벅적', '맛있다', '맛집', '쏘쏘', '나쁘다', '서민적인']",['파전']
20501,S0034704,점심식사분위기고급스러운 예쁜 푸짐한편의시설무료주차매우만족만족보통불만매우불만평가참여를...,['점심식사'],"['맛집', '많이', '예쁘다', '고급']",None


In [43]:
table.to_csv('DGY_리뷰테이블5.csv', index=False, encoding='utf-8-sig')